In [37]:
# generate random x_train, x_bndry values
import numpy as np
import tensorflow as tf
np.random.seed(123)

nI, nB = (3, 3)

x_train = np.random.rand(nI,2)
x_bndry = np.random.rand(nB,2)

In [38]:
# define kernels for tensorflow method
L = 0.2
import tensorflow as tf
import matplotlib.pyplot as plt
DTYPE = tf.float64

def kernel(x, xp):
    xbar = (x - xp) / L
    print(f"{xbar.shape=}")
    return tf.exp(-0.5 * tf.reduce_sum(tf.pow(xbar, 2.0), axis=-1))

def kernel_lapl(x, xp):
    xbar = tf.linalg.norm(x - xp, axis=-1)/L
    print(f"{xbar=}")
    K = kernel(x, xp)
    return K * (xbar**2 - 2.0) / L**2

def kernel_lapl2(x, xp):
    xbar = tf.reduce_sum((x - xp) / L, axis=-1)
    K = kernel(x, xp)
    return K * (xbar**4 - 8 * xbar**2 + 8) / L**4

x_all_L = tf.expand_dims(x_train, axis=1)
x_interior_R = tf.expand_dims(x_bndry, axis=0)
K12_tf = tf.constant(kernel_lapl(x_all_L, x_interior_R), dtype=DTYPE)
# plt.imshow(K12_tf)
# plt.colorbar()

xbar=<tf.Tensor: shape=(3, 3), dtype=float64, numpy=
array([[2.44835502, 1.20091315, 2.83277359],
       [3.8282198 , 1.49917176, 1.06209201],
       [1.84915028, 1.20270786, 2.4248513 ]])>
xbar.shape=TensorShape([3, 3, 2])


In [39]:
# define kernels my old python style
G_sigma = 0.2
eta = 1e-5
print("Building K(x,phi) and K(phi,phi) matrices...")
def kernel(xm, xl, sigma=1, order=0):
    D = xm.shape[0]
    norm_sq = np.dot(xm-xl, xm-xl)
    if order == 0: # regular function
        return np.exp(-norm_sq/2.0/sigma**2)
    elif order == 1: # first Laplacian
        K = kernel(xm,xl,sigma=sigma, order=0)
        return 1 * K * (norm_sq - D * sigma**2) / sigma**4
    elif order == 2:
        K = kernel(xm,xl,sigma=sigma, order=0)
        return K * (norm_sq**2 - 2 * (2 + D) * norm_sq * sigma**2 + D * (2 + D) * sigma**4) / sigma**8
    
X = np.concatenate([x_train, x_bndry, x_train], axis=0)

K12_old = np.zeros((nI, nB))
for m in range(nI):
    xm = x_train[m,:]
    for l in range(nB):
        xl = x_bndry[l,:]
        # l_2nd = l >= nB + nI
        # deriv_order = 1.0 * (m_2nd + l_2nd)
        # double check the sign stuff here
        # sgn1 = 1.0 if deriv_order == 1.0 else 1.0
        # sgn2 = 1.0 if l_2nd else 1.0
        print(f"{xm=} {xl=}")
        K12_old[m,l] = kernel(xm,xl,sigma=G_sigma, order=1)

# plt.imshow(K12_old)
# plt.colorbar()
rel_err = abs((K12_tf - K12_old) / K12_old)
print(f"{K12_tf=}")
print(f"{K12_old=}")
print(f"{rel_err=}")

Building K(x,phi) and K(phi,phi) matrices...
xm=array([0.69646919, 0.28613933]) xl=array([0.9807642 , 0.68482974])
xm=array([0.69646919, 0.28613933]) xl=array([0.4809319 , 0.39211752])
xm=array([0.69646919, 0.28613933]) xl=array([0.34317802, 0.72904971])
xm=array([0.22685145, 0.55131477]) xl=array([0.9807642 , 0.68482974])
xm=array([0.22685145, 0.55131477]) xl=array([0.4809319 , 0.39211752])
xm=array([0.22685145, 0.55131477]) xl=array([0.34317802, 0.72904971])
xm=array([0.71946897, 0.42310646]) xl=array([0.9807642 , 0.68482974])
xm=array([0.71946897, 0.42310646]) xl=array([0.4809319 , 0.39211752])
xm=array([0.71946897, 0.42310646]) xl=array([0.34317802, 0.72904971])
K12_tf=<tf.Tensor: shape=(3, 3), dtype=float64, numpy=
array([[  4.98562436,  -6.78041586,   2.72488122],
       [  0.2079029 ,   2.0114134 , -12.40184219],
       [  6.41989159,  -6.7134839 ,   5.12809519]])>
K12_old=array([[  4.98562436,  -6.78041586,   2.72488122],
       [  0.2079029 ,   2.0114134 , -12.40184219],
     